In [1]:
import pandas as pd
import os

Before running this notebook, download the data repositiory from http://github.com/jump-cellpainting/2023_Chandrasekaran_submitted.git

In [2]:
# Read experiment metadata

batch = "2020_11_04_CPJUMP1"

experiment_df = (
    pd.read_csv('../2023_Chandrasekaran_submitted/benchmark/output/experiment-metadata.tsv', sep='\t')
    .query("Batch==@batch")
    .query("Density==100")
    .query('Antibiotics=="absent"')
)

experiment_df.drop(
    experiment_df[
        (experiment_df.Perturbation == "compound") & (experiment_df.Cell_line == "Cas9")
    ].index,
    inplace=True,
)

In [3]:
experiment_df.head()

,Batch,Plate_Map_Name,Assay_Plate_Barcode,Perturbation,Cell_type,Time,Density,Antibiotics,Cell_line,Time_delay,Times_imaged,Anomaly,Number_of_images
0,2020_11_04_CPJUMP1,JUMP-Target-1_compound_platemap,BR00116991,compound,A549,24,100,absent,Parental,Day0,1,WGA,27648
1,2020_11_04_CPJUMP1,JUMP-Target-1_compound_platemap,BR00116992,compound,A549,24,100,absent,Parental,Day0,1,WGA,27640
2,2020_11_04_CPJUMP1,JUMP-Target-1_compound_platemap,BR00116993,compound,A549,24,100,absent,Parental,Day0,1,WGA,27352
3,2020_11_04_CPJUMP1,JUMP-Target-1_compound_platemap,BR00116994,compound,A549,24,100,absent,Parental,Day0,1,WGA,27576
4,2020_11_04_CPJUMP1,JUMP-Target-1_compound_platemap,BR00116995,compound,U2OS,24,100,absent,Parental,Day0,1,WGA,27648


In [4]:
# Read all the plates

all_plates = pd.DataFrame()

for plate in experiment_df.Assay_Plate_Barcode.unique():
    df = (
        pd.read_csv(f'../2023_Chandrasekaran_submitted/profiles/{batch}/{plate}/{plate}_normalized_feature_select_negcon_batch.csv.gz')
        .assign(Metadata_Perturbation = experiment_df.query('Assay_Plate_Barcode==@plate').Perturbation.values[0])
        .assign(Metadata_Cell_type = experiment_df.query('Assay_Plate_Barcode==@plate').Cell_type.values[0])
        .assign(Metadata_Time = experiment_df.query('Assay_Plate_Barcode==@plate').Time.values[0])
    )

    all_plates = pd.concat(
        [all_plates, df],
        axis=0,
        ignore_index=True,
    )

In [5]:
# Write to parquet

if not os.path.isdir('../compare_parquet'):
    os.mkdir('../compare_parquet')
    all_plates.to_parquet('../compare_parquet/data.parquet', index=False)